## MAIN

In [209]:
# from google.colab import drive
# drive.mount('/content/drive')

In [210]:
%ls
%pwd

 Volume in drive D is DATA
 Volume Serial Number is EADF-1FDB

 Directory of d:\SeM\ML\190394R_Lab01

10/02/2023  01:58 PM    <DIR>          .
10/02/2023  01:58 PM    <DIR>          ..
10/02/2023  02:00 PM            32,949 190394R_Lab01.ipynb
10/02/2023  11:47 AM             9,954 190394R_lab01.py
08/26/2023  04:21 AM           521,052 190394R_Lab01_Report.pdf
10/02/2023  01:58 PM    <DIR>          output
09/18/2023  10:25 AM               409 README.md
08/25/2023  11:46 PM         2,382,718 test.csv
08/24/2023  03:05 PM       142,453,947 train.csv
08/25/2023  11:05 AM         2,387,396 valid.csv
               7 File(s)    147,788,425 bytes
               3 Dir(s)  303,195,418,624 bytes free


'd:\\SeM\\ML\\190394R_Lab01'

In [211]:
# %cd '/content/drive/MyDrive/lab_01'

In [212]:
import pandas as pd
import numpy as np

# Labes
L1 = 'label_1'
L2 = 'label_2'
L3 = 'label_3'
L4 = 'label_4'

LABELS = [L1,L2,L3,L4]
AGE_LABEL = L2
FEATURES = [f"feature_{i}" for i in range (1,257)]

Import csv files


In [213]:
train_df = pd.read_csv("train.csv")
valid_df = pd.read_csv("valid.csv")
test_df = pd.read_csv("test.csv")

Data segmentation and drop null values

In [214]:
from sklearn.preprocessing import StandardScaler
x_train = {}
x_valid = {}
y_train = {}
y_valid = {}
x_test = {}

for target_label in LABELS:
  tr_df = train_df[train_df['label_2'].notna()] if target_label == 'label_2' else train_df
  vl_df = valid_df[valid_df['label_2'].notna()] if target_label == 'label_2' else valid_df
  tst_df = test_df
  scaler = StandardScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(tr_df.drop(LABELS, axis = 1)), columns=FEATURES)
  y_train[target_label] = tr_df[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(vl_df.drop(LABELS, axis = 1)), columns=FEATURES)
  y_valid[target_label] = vl_df[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(tst_df.drop(LABELS, axis = 1)), columns=FEATURES)

# Label_1

In [215]:
from sklearn import svm

clf = svm.SVC(kernel='linear', class_weight='balanced')
clf.fit(x_train[L1], y_train[L1])

SVC(class_weight='balanced', kernel='linear')

In [216]:
from sklearn import metrics

In [217]:
y_pred = clf.predict(x_valid[L1])
y_pred_test_before_L1 = clf.predict(x_test[L1])

## Accuracy Before Feature Engineering

In [218]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L1], y_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L1], y_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L1], y_pred,average='weighted')}")

Accuracy: 0.9906666666666667
Precision: 0.9914608132608133
Recall: 0.9906666666666667


Feature Selection using SelectKBest


In [219]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=100)
x_new = selector.fit_transform(x_train[L1], y_train[L1])

In [220]:

print(x_new.shape)

(28520, 100)


In [221]:
clf = svm.SVC(kernel='linear' )
clf.fit(x_new, y_train[L1])

SVC(kernel='linear')

In [222]:
y_new_pred = clf.predict(selector.transform(x_valid[L1]))

## Accuracy After Feature Engineering - SelectKBest

In [223]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L1], y_new_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L1], y_new_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L1], y_new_pred,average='weighted')}")

Accuracy: 0.976
Precision: 0.9774185666185666
Recall: 0.976


In [224]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95, svd_solver ='full')
pca.fit(x_train[L1])

PCA(n_components=0.95, svd_solver='full')

In [225]:

x_train_trans = pd.DataFrame(pca.transform(x_train[L1]))
x_valid_trans = pd.DataFrame(pca.transform(x_valid[L1]))
x_test_pca = pca.transform(x_test[L1])

print(x_train_trans.shape, x_valid_trans.shape)

(28520, 67) (750, 67)


In [226]:
clf = svm.SVC(kernel='linear' )
clf.fit(x_train_trans, y_train[L1])

SVC(kernel='linear')

In [227]:
y_new_trans = clf.predict(x_valid_trans)

In [228]:
y_pred_test_after_L1 = clf.predict(x_test_pca)
print('Predicted labels after feature engineering:', y_pred_test_after_L1)

Predicted labels after feature engineering: [45 45 45 45 45 45  5  5  5  5  5  5  5  5  5  5  5  5 60 60 60 60 60 60
 60 60 19 19 19 19 19 19 19 19 19 19 39 19 19 19 19 19 11 11 11 11 11 11
 11 11 11 11 11 11 52 36 52 52 52 52 52 52 52 52 52 25 25 25 25 25 25 25
 25 25 25 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 51
 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 35 35 35 35 35 35 35 35
 56 56 56 56 56 56 56 56 56 56 56 53 53 53 53 53 55 22 53 53 53  3  3  3
  3  3  3  3  3  3  3 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 43 43
 43 43 43 43 43 43 43 43 43 36 58 58 58 58 58 58 58 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 37 37 37 37 37 37 37 37 37 37 37 37 14 37 37 37 37
 55 55 55 55 55 55 55 55 55 55 55 55 55 55 55 55 55 17  2 17 24 17 17 17
 17 17 17 17 17 17  2  2  2  2  2  2  2  2  2  2  2  2 47 47 47 47 47 47
 47 47 47 26 47 47 47 54 54 54 54 54 54 54 54 54 54 54 21 21 21 21 21 21
 21 21 21 21 21 21 34 27 34 34 34 34 34 34 34 23 23 23 23 23 23 23 23 23
 23 23 

selected features writing

In [229]:
output_df = pd.DataFrame({
    'Predicted labels before feature engineering': y_pred_test_before_L1,
    'Predicted labels after feature engineering': y_pred_test_after_L1,
    'No. of new features': x_test_pca.shape[1]
})


for i in range(x_test_pca.shape[1]):
  output_df[f'New feature {i+1}'] = x_test_pca[:, i]

output_df.head()

,Predicted labels before feature engineering,Predicted labels after feature engineering,No. of new features,New feature 1,New feature 2,New feature 3,New feature 4,New feature 5,New feature 6,New feature 7,...,New feature 58,New feature 59,New feature 60,New feature 61,New feature 62,New feature 63,New feature 64,New feature 65,New feature 66,New feature 67
0,45,45,67,-1.967567,6.945703,-3.947321,-1.108198,-6.440248,-1.053785,3.471608,...,0.578949,-1.740188,-0.118466,1.373034,1.198465,0.701518,-0.092956,-0.961733,-1.577341,0.453901
1,45,45,67,-6.692310,8.884467,-0.489867,-10.240206,-4.576115,2.229018,-2.025519,...,-0.430111,1.420051,-0.984095,0.078896,-0.163620,1.022940,1.326354,0.892220,-0.054520,-0.060989
2,45,45,67,-4.810155,3.547653,-0.297522,-5.550987,-2.225314,-1.305231,-0.434514,...,-0.623699,-0.565450,-0.924432,-0.114114,-0.526053,-0.612581,0.770387,0.258546,-0.176509,-0.049412
3,45,45,67,-2.185614,5.993099,-1.089448,-0.938549,2.566186,4.411621,0.475160,...,0.828342,-0.924356,0.728764,-0.928061,0.356018,0.039349,-1.203948,0.575542,1.194812,-0.065328
4,45,45,67,0.695696,6.181181,1.442544,-3.112025,-2.881196,-1.171098,1.626019,...,-0.643171,0.046833,-0.774574,0.500928,0.716768,-0.848316,0.289207,1.111349,0.781681,0.461655


In [230]:
# Save the DataFrame to the specified CSV file path
output_df.to_csv(f"output/190394R_label_1.csv", index=False)

## Accuracy After Feature Engineering - PCA

In [231]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L1], y_new_trans)}")
print(f"Precision: {metrics.precision_score(y_valid[L1], y_new_trans, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L1], y_new_trans,average='weighted')}")

Accuracy: 0.9813333333333333
Precision: 0.9835733337182255
Recall: 0.9813333333333333


# Label_2

In [232]:
from sklearn.neighbors import KNeighborsClassifier
k = 5  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

In [233]:
knn_classifier.fit(x_train[L2], y_train[L2])

KNeighborsClassifier()

In [234]:
x_test[L2].shape

(750, 256)

In [235]:
y_pred = knn_classifier.predict(np.array(x_valid[L2]))
y_pred_test_before_L2 = knn_classifier.predict(np.array(x_test[L2]))

c:\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


## Accuracy Before Feature Engineering

In [236]:
from sklearn import metrics
print(f"Accuracy: {metrics.accuracy_score(y_valid[L2], y_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L2], y_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L2], y_pred,average='weighted')}")

Accuracy: 0.9877717391304348
Precision: 0.9880671083222579
Recall: 0.9877717391304348


In [237]:
#Feature Selection
pca = PCA(n_components = 0.85, svd_solver ='full')
pca.fit(x_train[L2])
x_train_trans = pd.DataFrame(pca.transform(x_train[L2]))
x_valid_trans = pd.DataFrame(pca.transform(x_valid[L2]))
x_test_pca = pca.transform(x_test[L2])

In [238]:
x_test_pca.shape

(750, 39)

In [239]:

print(x_train_trans.shape, x_valid_trans.shape)

(28040, 39) (736, 39)


In [240]:
knn_classifier.fit(x_train_trans, y_train[L2])

KNeighborsClassifier()

In [241]:
y_pred = knn_classifier.predict(np.array(x_valid_trans))

In [242]:
y_pred_test_after_L2 = knn_classifier.predict(x_test_pca)
print('Predicted labels after feature engineering:', y_pred_test_after_L2)

Predicted labels after feature engineering: [26. 31. 30. 27. 31. 23. 25. 25. 25. 25. 25. 25. 25. 25. 25. 25. 25. 25.
 27. 27. 27. 27. 27. 27. 27. 22. 23. 23. 23. 23. 23. 23. 23. 23. 23. 23.
 23. 23. 26. 23. 23. 23. 33. 33. 33. 33. 33. 33. 33. 33. 33. 33. 33. 33.
 34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 22. 22. 22. 22. 22. 22. 22.
 22. 22. 22. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26.
 26. 26. 26. 26. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 33. 24. 24. 31. 31. 31.
 31. 31. 31. 31. 31. 31. 31. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26.
 26. 26. 26. 26. 31. 31. 31. 31. 31. 31. 31. 31. 31. 31. 31. 29. 29. 29.
 29. 29. 29. 29. 29. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 27. 27. 27. 27. 27. 27. 27. 27. 27. 27. 27. 27. 31. 27. 27. 27. 27.
 23. 23. 23. 23. 23. 23. 23. 23. 23. 23. 23. 28. 26. 23. 23. 26. 23. 26.
 26. 26

In [243]:
y_pred_test_before_L2.shape, y_pred_test_after_L2.shape

((750,), (750,))

In [244]:
output_df = pd.DataFrame({
    'Predicted labels before feature engineering': y_pred_test_before_L2,
    'Predicted labels after feature engineering': y_pred_test_after_L2,
    'No. of new features': x_test_pca.shape[1]
})


for i in range(x_test_pca.shape[1]):
  output_df[f'New feature {i+1}'] = x_test_pca[:, i]

output_df.head()

,Predicted labels before feature engineering,Predicted labels after feature engineering,No. of new features,New feature 1,New feature 2,New feature 3,New feature 4,New feature 5,New feature 6,New feature 7,...,New feature 30,New feature 31,New feature 32,New feature 33,New feature 34,New feature 35,New feature 36,New feature 37,New feature 38,New feature 39
0,26.0,26.0,39,-1.812223,6.687155,-3.641918,-0.345455,-6.543906,-0.232424,3.680529,...,-2.329460,1.562674,0.135665,1.988316,2.905189,1.233378,-1.572804,-0.391497,-2.491533,2.465499
1,31.0,31.0,39,-6.429202,8.208834,-0.228147,8.026291,-6.485308,4.389094,-1.675062,...,-0.278233,1.361253,0.039520,-1.158711,-0.013061,0.241287,-0.949262,-1.418688,-1.367894,1.490290
2,25.0,30.0,39,-4.636527,3.081759,-0.201630,4.513831,-3.444809,-0.378538,-0.598476,...,-1.208152,3.051534,-0.314827,-0.773049,-1.442211,0.534809,-1.197135,-1.295337,0.562543,1.769142
3,27.0,27.0,39,-2.033135,5.574049,-0.977140,0.732895,2.986661,4.048778,0.064816,...,-2.291276,1.777321,-1.438575,-2.084518,-2.807757,0.507345,0.325432,-1.894801,0.626920,1.327646
4,31.0,31.0,39,0.862663,5.753936,1.644088,2.243056,-3.438429,-0.409911,1.583930,...,-2.007713,-0.161983,0.644221,-1.585627,0.040405,0.023946,-0.025959,-2.123518,-0.441444,2.844272


In [245]:
# Save the DataFrame to the specified CSV file path
output_df.to_csv(f"output/190394R_label_2.csv", index=False)

## Accuracy After Feature Engineering - PCA

In [246]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L2], y_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L2], y_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L2], y_pred,average='weighted')}")

Accuracy: 0.9823369565217391
Precision: 0.9829790686548117
Recall: 0.9823369565217391


# Label_3

In [247]:
clf = svm.SVC(kernel='linear', class_weight='balanced')
clf.fit(x_train[L3], y_train[L3])

SVC(class_weight='balanced', kernel='linear')

In [248]:
y_pred = clf.predict(x_valid[L3])
y_pred_test_before_L3 = clf.predict(x_test[L3])


## Accuracy Before Feature Engineering

In [249]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L3], y_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L3], y_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L3], y_pred,average='weighted')}")

Accuracy: 0.9986666666666667
Precision: 0.9986759906759908
Recall: 0.9986666666666667


In [250]:
selector = SelectKBest(f_classif, k=15)
x_new = selector.fit_transform(x_train[L3], y_train[L3])

In [251]:
print(x_new.shape)

(28520, 15)


In [252]:
clf = svm.SVC(kernel='linear' )
clf.fit(x_new, y_train[L3])

SVC(kernel='linear')

In [253]:
y_new_pred = clf.predict(selector.transform(x_valid[L3]))

## Accuracy After Feature Engineering - selectKBest

In [254]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L3], y_new_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L3], y_new_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L3], y_pred,average='weighted')}")

Accuracy: 0.984
Precision: 0.9841076274294095
Recall: 0.9986666666666667


In [255]:
pca = PCA(n_components = 0.45, svd_solver ='full')

In [256]:
pca = PCA(n_components = 0.45, svd_solver ='full')
pca.fit(x_train[L3])
x_train_trans = pd.DataFrame(pca.transform(x_train[L3]))
x_valid_trans = pd.DataFrame(pca.transform(x_valid[L3]))
x_test_pca = pca.transform(x_test[L3])

print(x_train_trans.shape, x_valid_trans.shape)

(28520, 9) (750, 9)


In [257]:
clf = svm.SVC(kernel='linear' )
clf.fit(x_train_trans, y_train[L3])

SVC(kernel='linear')

In [258]:
y_new_trans = clf.predict(x_valid_trans)

In [259]:
y_pred_test_after_L3 = clf.predict(x_test_pca)
# print('Predicted labels after feature engineering:', y_pred_test_after_L3)

In [260]:
output_df = pd.DataFrame({
    'Predicted labels before feature engineering': y_pred_test_before_L3,
    'Predicted labels after feature engineering': y_pred_test_after_L3,
    'No. of new features': x_test_pca.shape[1]
})


for i in range(x_test_pca.shape[1]):
  output_df[f'New feature {i+1}'] = x_test_pca[:, i]

output_df.head()

,Predicted labels before feature engineering,Predicted labels after feature engineering,No. of new features,New feature 1,New feature 2,New feature 3,New feature 4,New feature 5,New feature 6,New feature 7,New feature 8,New feature 9
0,1,1,9,-1.967567,6.945703,-3.947321,-1.108198,-6.440248,-1.053785,3.471608,2.685400,5.946839
1,1,1,9,-6.692310,8.884467,-0.489867,-10.240206,-4.576115,2.229018,-2.025519,2.366489,-2.672080
2,1,1,9,-4.810155,3.547653,-0.297522,-5.550987,-2.225314,-1.305231,-0.434514,-0.250999,-5.807706
3,1,1,9,-2.185614,5.993099,-1.089448,-0.938549,2.566186,4.411621,0.475160,0.404839,1.420231
4,1,1,9,0.695696,6.181181,1.442544,-3.112025,-2.881196,-1.171098,1.626019,3.521929,0.719271


In [261]:
# Save the DataFrame to the specified CSV file path
output_df.to_csv(f"output/190394R_label_3.csv", index=False)

Accuracy After Feature Engineering - PCA

In [262]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L3], y_new_trans)}")
print(f"Precision: {metrics.precision_score(y_valid[L3], y_new_trans, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L3], y_new_trans,average='weighted')}")

Accuracy: 0.9866666666666667
Precision: 0.986616705698673
Recall: 0.9866666666666667


# Label_4

In [263]:
k = 5  # Number of neighbors
knn_classifier4 = KNeighborsClassifier(n_neighbors=k)

In [264]:
knn_classifier4.fit(x_train[L4], y_train[L4])

KNeighborsClassifier()

In [265]:
y_pred = knn_classifier4.predict(np.array(x_valid[L4]))
y_pred_test_before_L4 = knn_classifier4.predict(np.array(x_test[L4]))

c:\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


## Accuracy Before Feature Engineering

In [266]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L4], y_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L4], y_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L4], y_pred,average='weighted')}")

Accuracy: 0.9946666666666667
Precision: 0.9947064676616915
Recall: 0.9946666666666667


In [267]:
pca = PCA(n_components = 0.85, svd_solver ='full')

In [268]:

pca.fit(x_train[L4])
x_train_trans = pd.DataFrame(pca.transform(x_train[L4]))
x_valid_trans = pd.DataFrame(pca.transform(x_valid[L4]))
x_test_pca = pca.transform(x_test[L4])

print(x_train_trans.shape, x_valid_trans.shape)

(28520, 39) (750, 39)


In [269]:
knn_classifier4.fit(x_train_trans, y_train[L4])

KNeighborsClassifier()

In [270]:
y_pred = knn_classifier4.predict(np.array(x_valid_trans))


In [271]:
y_pred_test_after_L4 = knn_classifier4.predict(x_test_pca)
print('Predicted labels after feature engineering:', y_pred_test_after_L4)

Predicted labels after feature engineering: [ 6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6 13 13 13 13 13 13
 13 13  4  4  4  4  4  4  4  4  4  4  4  4  6  4  4  4  6  6  6  6  6  6
  6  6  6  6  6  6  5  5  5  5  5  5  5  5  5  5  5  1  1  1  1  1  1  1
  1  1  1  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  2  2  2  2  2  2  2  2
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7 12  7  7  7  7
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  3  3  3  3  3  3
  3  3  3  3  3  3  3  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6 

In [272]:
output_df = pd.DataFrame({
    'Predicted labels before feature engineering': y_pred_test_before_L4,
    'Predicted labels after feature engineering': y_pred_test_after_L4,
    'No. of new features': x_test_pca.shape[1]
})


for i in range(x_test_pca.shape[1]):
  output_df[f'New feature {i+1}'] = x_test_pca[:, i]

output_df.head()

,Predicted labels before feature engineering,Predicted labels after feature engineering,No. of new features,New feature 1,New feature 2,New feature 3,New feature 4,New feature 5,New feature 6,New feature 7,...,New feature 30,New feature 31,New feature 32,New feature 33,New feature 34,New feature 35,New feature 36,New feature 37,New feature 38,New feature 39
0,6,6,39,-1.967567,6.945703,-3.947321,-1.108198,-6.440248,-1.053785,3.471608,...,1.833450,1.251084,0.233115,2.702653,2.902344,-1.011773,-1.376811,-0.408022,2.432256,-1.071824
1,6,6,39,-6.692310,8.884467,-0.489867,-10.240206,-4.576115,2.229018,-2.025519,...,0.115382,0.855785,0.174110,-0.987622,0.365215,-0.650891,-1.446016,-1.175507,0.982672,-0.651031
2,6,6,39,-4.810155,3.547653,-0.297522,-5.550987,-2.225314,-1.305231,-0.434514,...,1.085848,2.371444,-0.099275,-0.838729,-1.036768,-1.143609,-1.294504,-0.763070,0.093595,1.043378
3,6,6,39,-2.185614,5.993099,-1.089448,-0.938549,2.566186,4.411621,0.475160,...,1.001001,1.175007,-1.125593,-2.486466,-2.091227,-1.359647,-0.514262,-2.105196,0.170947,1.221653
4,6,6,39,0.695696,6.181181,1.442544,-3.112025,-2.881196,-1.171098,1.626019,...,2.235556,-0.614469,0.779440,-1.510207,0.395952,-0.164901,-0.709025,-1.933551,0.545245,0.083859


In [273]:
# Save the DataFrame to the specified CSV file path
output_df.to_csv(f"output/190394R_label_4.csv", index=False)

In [274]:
y_pred_test_after = knn_classifier4.predict(x_test_pca)

Accuracy After Feature Engineering - PCA

In [275]:
print(f"Accuracy: {metrics.accuracy_score(y_valid[L4], y_pred)}")
print(f"Precision: {metrics.precision_score(y_valid[L4], y_pred, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L4], y_pred,average='weighted')}")

Accuracy: 0.988
Precision: 0.9882956567537573
Recall: 0.988
